In [1]:
import pandas as pd
import geohash as gh
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime, timedelta
from multiprocessing import  Pool

In [2]:
def Griding(df, start, end):
    gb = df.groupby('geoHash')
    # idx = pd.date_range(start='6/1/2020', end = '7/1/2020', freq='15T')
    idx = pd.date_range(start=start, end=end, freq='15T')
    output_list = []
    for key in tqdm_notebook(gb.groups.keys()):
        # tmp_df = gb.get_group(key).set_index('CallTime').resample('15T').count()['JobID']
        tmp_df = gb.get_group(key).resample('15T').count()['JobID']
        tmp_df.name = key
        tmp_df = tmp_df.reindex(idx, fill_value=0)
        output_list.append(tmp_df)
    return output_list


def NoiseCheck(dt, df):
    if dt['CustPhone'] in CommercialTel:
        return dt['CustPhone'] in df[str(dt.name - timedelta(minutes=1)):str(dt.name - timedelta(seconds=1))]['CustPhone'].values
        # return False
    else:
        return dt['CustPhone'] in df[str(dt.name - timedelta(minutes=15)):str(dt.name - timedelta(seconds=1))]['CustPhone'].values
    

def ApplyNoiseCheck(sub_df, df):
    return sub_df.apply(lambda x: NoiseCheck(x, df), axis=1)


def parallelize_dataframe(df, func, n_cores=31):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.starmap(func, [(df_split[n], df_split[n]) for n in range(n_cores)]))
    pool.close()
    pool.join()
    return df

In [3]:
%%time

CommercialTel = pd.read_csv('data/CommercialTel.csv').squeeze().values

input_path = 'Job_GPS_20200601_20200630.csv'
df = pd.read_csv(input_path, parse_dates=['CallTime'], index_col='CallTime')
df = df.dropna()
df = df.sort_values(by='CallTime')
df

<string>:2: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 7.06 s, sys: 1.14 s, total: 8.2 s
Wall time: 8.19 s


,JobID,CustPhone,Lng_X,Lat_Y
CallTime,,,,
2020-06-01 00:00:00,DIP20060100001,B02BBB6C0CBD066956A7984BC4B6D7349A80D853,120.306564,22.686715
2020-06-01 00:00:00,TIP20060100001,9B8442E59E5347C3EB23C225840F1394AC7935EC,121.509806,25.000988
2020-06-01 00:00:03,APS20060100001,CFBDC9749F31C2C9108222E5CBBACAFCF61DD4C1,121.712547,25.126773
2020-06-01 00:00:06,DIP20060100002,C028A3894464E64026C28D927A166086222446D7,121.519796,25.007494
2020-06-01 00:00:08,16820060100001,18A44CD4617C7B8955DA2566661EEC23475EB305,121.501663,25.073829
...,...,...,...,...
2020-06-30 23:59:54,DAP2006300J115,F7AE8676D2EB78F03E88DCDA5A76A0402938DEF2,121.588523,25.086883
2020-06-30 23:59:55,71120063005771,12B1BA409EE79F0D7B7AE11CE20ADEEF2121A630,121.475875,25.015148
2020-06-30 23:59:58,DIP2006301C783,F45B42D107983F103E1C2690DBAB640109670E8C,121.567152,25.041042


In [4]:
%%time
res = parallelize_dataframe(df, ApplyNoiseCheck)

df['Noise'] = res
new_df.to_csv('Job_GPS_20200601_20200630_noisecheck_15_1.csv')
df.head()

CPU times: user 3.63 s, sys: 1.93 s, total: 5.56 s
Wall time: 7min 2s


,JobID,CustPhone,Lng_X,Lat_Y,Noise
CallTime,,,,,
2020-06-01 00:00:00,DIP20060100001,B02BBB6C0CBD066956A7984BC4B6D7349A80D853,120.306564,22.686715,False
2020-06-01 00:00:00,TIP20060100001,9B8442E59E5347C3EB23C225840F1394AC7935EC,121.509806,25.000988,False
2020-06-01 00:00:03,APS20060100001,CFBDC9749F31C2C9108222E5CBBACAFCF61DD4C1,121.712547,25.126773,False
2020-06-01 00:00:06,DIP20060100002,C028A3894464E64026C28D927A166086222446D7,121.519796,25.007494,False
2020-06-01 00:00:08,16820060100001,18A44CD4617C7B8955DA2566661EEC23475EB305,121.501663,25.073829,False


In [15]:
%%time
new_df = df[df['Noise'] == 0]
new_df['geoHash'] = new_df.apply(lambda x: gh.encode(x['Lat_Y'], x['Lng_X'], precision=6),axis=1)
new_df.head()

,JobID,CustPhone,Lng_X,Lat_Y,Noise,geoHash
CallTime,,,,,,
2020-06-01 00:00:00,DIP20060100001,B02BBB6C0CBD066956A7984BC4B6D7349A80D853,120.306564,22.686715,False,wsj919
2020-06-01 00:00:00,TIP20060100001,9B8442E59E5347C3EB23C225840F1394AC7935EC,121.509806,25.000988,False,wsqqjp
2020-06-01 00:00:03,APS20060100001,CFBDC9749F31C2C9108222E5CBBACAFCF61DD4C1,121.712547,25.126773,False,wsqwcw
2020-06-01 00:00:06,DIP20060100002,C028A3894464E64026C28D927A166086222446D7,121.519796,25.007494,False,wsqqm2
2020-06-01 00:00:08,16820060100001,18A44CD4617C7B8955DA2566661EEC23475EB305,121.501663,25.073829,False,wsqqsu


In [12]:
%%time
output_list = Griding(new_df, '6/1/2020', '7/1/2020')
final_df = pd.concat(output_list,axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



CPU times: user 30.3 s, sys: 852 ms, total: 31.1 s
Wall time: 30.9 s


In [13]:
final_df

,wevufc,wevug1,wevug3,wevug6,wevug7,wevug8,wevugd,wevuge,wevugh,wevugm,...,wsqx52,wsqx53,wsqx54,wsqx55,wsqx58,wsqx59,wsqx80,wsqx81,wsqy0p,wsqy21
2020-06-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 00:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2020-06-01 00:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30 23:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-30 23:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-30 23:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-30 23:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
%%time
MLdata = pd.read_csv('data/acceptance_15_1.csv', nrows=1, index_col=0)
final_df = final_df.reindex(columns=MLdata.columns, fill_value=0)
final_df.head()

CPU times: user 800 ms, sys: 192 ms, total: 992 ms
Wall time: 982 ms


,wsmb3r,wsq741,wsm8tk,wsje3q,wsqjzc,wsqr5y,wsqq9e,wsqnr3,wsm844,wsqwch,...,wsmgn5,wsjtgy,wsq2w6,wsj7je,wsqqzj,wsqw2q,wsjzfe,wsmb1r,wsqm79,wevxwc
2020-06-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 00:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 00:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-06-01 01:00:00,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
final_df.to_csv('acceptance_15_1_20200601_20200630.csv')